In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import SGD

In [ ]:
df = pd.read_csv('/kaggle/input/network-slicing-in-5g/train_dataset.csv')
df.head()

,LTE/5g Category,Time,Packet Loss Rate,Packet delay,IoT,LTE/5G,GBR,Non-GBR,AR/VR/Gaming,Healthcare,Industry 4.0,IoT Devices,Public Safety,Smart City & Home,Smart Transportation,Smartphone,slice Type
0,14,0,0.000001,10,1,0,0,1,0,0,0,0,1,0,0,0,3
1,18,20,0.001000,100,0,1,1,0,1,0,0,0,0,0,0,0,1
2,17,14,0.000001,300,0,1,0,1,0,0,0,0,0,0,0,1,1
3,3,17,0.010000,100,0,1,0,1,0,0,0,0,0,0,0,1,1
4,9,4,0.010000,50,1,0,0,1,0,0,0,0,0,1,0,0,2


In [ ]:
#The idea behind StandardScaler is that it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.
#Given the distribution of the data, each value in the dataset will have the mean value subtracted, and then divided by the standard deviation of the whole dataset (or feature in the multivariate case).
scaler = StandardScaler().fit(df[['Packet Loss Rate','Packet delay','slice Type','Time','LTE/5g Category']].values)
transformed_dataset = scaler.transform(df[['Packet Loss Rate','Packet delay','slice Type','Time','LTE/5g Category']].values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()

,0,1,2,3,4
0,-0.708653,-0.979378,1.577084,-1.659519,0.499720
1,-0.478678,-0.132871,-0.853218,1.232521,1.160490
2,-0.708653,1.748255,-0.853218,0.364909,0.995297
3,1.593169,-0.132871,-0.853218,0.798715,-1.317397
4,1.593169,-0.603152,0.361933,-1.081111,-0.326242


In [ ]:
number_of_rows= df.values.shape[0] #all rows
window_length = 2 #sliding window
number_of_features = 5 #columns count

In [ ]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)

label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)
# create minibatches of 2 rows as element in train array
for i in range(0, number_of_rows-window_length):
    train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
    label[i]=transformed_df.iloc[i+window_length: i+window_length+1, 0: number_of_features]

In [ ]:
train.shape

(31581, 2, 5)

In [ ]:
label.shape

(31581, 5)

In [ ]:
train[0]

array([[-0.7086526 , -0.97937771,  1.57708365, -1.65951906,  0.49971983],
       [-0.47867759, -0.13287095, -0.85321803,  1.23252121,  1.1604896 ]])

In [ ]:
train[1]

array([[-0.47867759, -0.13287095, -0.85321803,  1.23252121,  1.1604896 ],
       [-0.7086526 ,  1.74825518, -0.85321803,  0.36490913,  0.99529716]])

In [ ]:
label[0]

array([-0.7086526 ,  1.74825518, -0.85321803,  0.36490913,  0.99529716])

In [ ]:
label[1]

array([ 1.59316934, -0.13287095, -0.85321803,  0.79871517, -1.31739706])

In [ ]:
batch_size = 25
model = Sequential()
# add 1st layer with 50 nodes/perceptons, each will process (2,5) matrix block
model.add(LSTM(50,
           input_shape=(window_length, number_of_features),
           return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,
           return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train, label, batch_size=32, epochs=8000, verbose=1)

Epoch 1/8000

48/48 [==============================] - 2s 38ms/step - loss: 0.9838 - acc: 0.1667

Epoch 2/8000

48/48 [==============================] - 0s 249us/step - loss: 0.9791 - acc: 0.1875

Epoch 3/8000

48/48 [==============================] - 0s 291us/step - loss: 0.9782 - acc: 0.1875

Epoch 4/8000

48/48 [==============================] - 0s 270us/step - loss: 0.9738 - acc: 0.1667

Epoch 5/8000

48/48 [==============================] - 0s 229us/step - loss: 0.9744 - acc: 0.2083

Epoch 6/8000

48/48 [==============================] - 0s 249us/step - loss: 0.9716 - acc: 0.1250

Epoch 7/8000

48/48 [==============================] - 0s 291us/step - loss: 0.9686 - acc: 0.1042

Epoch 8/8000

48/48 [==============================] - 0s 270us/step - loss: 0.9657 - acc: 0.1250

Epoch 9/8000

48/48 [==============================] - 0s 249us/step - loss: 0.9595 - acc: 0.1458

Epoch 10/8000

48/48 [==============================] - 0s 395us/step - loss: 0.9588 - acc: 0.1667

Epoch 11/8

In [1]:
# adam ; 8000 epochs
to_predict=np.array([[6,7,28,59,14,7],
[8,13,68,46,35,16]])
scaled_to_predict = scaler.transform(to_predict)
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

NameError: ignored

In [ ]:
# adam ; 8000 epochs
to_predict=np.array([[8,13,68,46,35,16],
[26,33,68,45,61,17]])
scaled_to_predict = scaler.transform(to_predict)
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

[46 37 20 23 45 15]
